In [ ]:
pip install geopandas folium matplotlib geopy

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import locale

## Geocodificação dos Endereços

Agora vamos utilizar o serviço de geocodificação para obter a latitude e longitude de cada imóvel.


In [ ]:
df = pd.read_csv('DecisaoImoveisLimpo.csv')

# Use Geopy para obter as coordenadas de latitude e longitude dos endereços
geolocator = Nominatim(user_agent='myGeocoder')

# Função para lidar com o GeocoderTimedOut
def do_geocode(address):
    try:
        return geolocator.geocode(address, timeout=15)
    except GeocoderTimedOut:
        return do_geocode(address)

# Cria uma nova coluna 'coords' com as coordenadas de cada imóvel
df['coords'] = df['Endereço'].apply(lambda x: do_geocode(x) if x else None)
df['coords'] = df['coords'].apply(lambda x: (x.latitude, x.longitude) if x else (None, None))

# Agora, 'coords' é uma coluna de tuplas, então vamos dividi-la em duas colunas: 'latitude' e 'longitude'
df[['latitude', 'longitude']] = pd.DataFrame(df['coords'].tolist(), index=df.index)

# Exclui a coluna 'coords' pois ela já não é mais necessária
df = df.drop(['coords'], axis=1)


## Criação do Mapa

Vamos utilizar a biblioteca Folium para criar um mapa interativo dos imóveis.


In [ ]:
# Define a localização para o Brasil
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

# Lendo os dados do mapa de bairros de Belo Horizonte
url = 'https://geoservicos.pbh.gov.br/geoserver/wfs?service=WFS&version=1.0.0&request=GetFeature&typeName=ide_bhgeo:BAIRRO&srsName=EPSG:31983&outputFormat=application%2Fjson'
gdf = gpd.read_file(url)

# Convertendo para lat-long
gdf = gdf.to_crs(epsg=4326)

# Converta o DataFrame em um GeoDataFrame
gdf_imoveis = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# Cria o mapa com os bairros de Belo Horizonte
x_map = gdf.geometry.centroid.x.mean()
y_map = gdf.geometry.centroid.y.mean()
mymap = folium.Map(location=[y_map, x_map], zoom_start=11, tiles='OpenStreetMap')

# Adiciona os polígonos dos bairros ao mapa
folium.GeoJson(gdf).add_to(mymap)

# Adiciona os pontos de cada imóvel ao mapa
for _, imovel in gdf_imoveis.iterrows():
    if not pd.isnull(imovel['latitude']) and not pd.isnull(imovel['longitude']):
        # Formata o preço com símbolo "R$", duas casas decimais e pontos de separação
        preco_formatado = locale.currency(imovel['Preço'], grouping=True)

        # Constrói o texto do popup com o preço formatado
        popup_text = f"Título: {imovel['Título']}<br> Endereço: {imovel['Endereço']}<br> Preço: {preco_formatado}"
        folium.Marker(location=[imovel['latitude'], imovel['longitude']],
                      popup=popup_text).add_to(mymap)

mymap.save('mapa.html')
